## Part 1: Preprocessing

In [16]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [17]:
# Determine the number of unique values in each column.
attrition_df.nunique()

,0
Age,43
Attrition,2
BusinessTravel,3
Department,3
DistanceFromHome,29
Education,5
EducationField,6
EnvironmentSatisfaction,4
HourlyRate,71
JobInvolvement,4


In [18]:
# Create y_df with the Attrition and Department columns

y_df = attrition_df[['Attrition', 'Department']]
print(y_df.head())

  Attrition              Department
0       Yes                   Sales
1        No  Research & Development
2       Yes  Research & Development
3        No  Research & Development
4        No  Research & Development


In [19]:
# Create a list of at least 10 column names to use as X data

X_data =['Education',
         'Age',
         'DistanceFromHome',
         'JobSatisfaction',
         'OverTime',
         'StockOptionLevel',
        'WorkLifeBalance',
         'YearsAtCompany',
         'YearsSinceLastPromotion',
         'NumCompaniesWorked']

# Create X_df using your selected columns

X_df = attrition_df[X_data]

# Show the data types for X_df

X_df.dtypes

,0
Education,int64
Age,int64
DistanceFromHome,int64
JobSatisfaction,int64
OverTime,object
StockOptionLevel,int64
WorkLifeBalance,int64
YearsAtCompany,int64
YearsSinceLastPromotion,int64
NumCompaniesWorked,int64


In [20]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_df,y_df)

In [21]:
print(X_df.columns)

Index(['Education', 'Age', 'DistanceFromHome', 'JobSatisfaction', 'OverTime',
       'StockOptionLevel', 'WorkLifeBalance', 'YearsAtCompany',
       'YearsSinceLastPromotion', 'NumCompaniesWorked'],
      dtype='object')


In [24]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
X_train['OverTime'].value_counts()

,count
OverTime,
No,795
Yes,307


In [25]:
# Fit the encoder on the training data
from sklearn.preprocessing import OneHotEncoder
OverTime_encoder = OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore')
OverTime_encoder.fit(np.array(X_train['OverTime']).reshape(-1, 1))
# Transform the training and test data
X_train['OverTimeEncoded'] = OverTime_encoder.transform(np.array(X_train['OverTime']).reshape(-1, 1))
X_test['OverTimeEncoded'] = OverTime_encoder.transform(np.array(X_test['OverTime']).reshape(-1, 1))
X_train = X_train.drop(columns='OverTime')
X_test = X_test.drop(columns='OverTime')
# Check the data types
print(X_train.dtypes)
print(X_test.dtypes)

Education                    int64
Age                          int64
DistanceFromHome             int64
JobSatisfaction              int64
StockOptionLevel             int64
WorkLifeBalance              int64
YearsAtCompany               int64
YearsSinceLastPromotion      int64
NumCompaniesWorked           int64
OverTimeEncoded            float64
dtype: object
Education                    int64
Age                          int64
DistanceFromHome             int64
JobSatisfaction              int64
StockOptionLevel             int64
WorkLifeBalance              int64
YearsAtCompany               int64
YearsSinceLastPromotion      int64
NumCompaniesWorked           int64
OverTimeEncoded            float64
dtype: object


In [27]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
X_scaler = scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [28]:
# Create a OneHotEncoder for the Department column
department_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore', drop=None)

# Fit the encoder to the training data
department_encoder.fit(np.array(y_train['Department']).reshape(-1, 1))

# Create two new variables by applying the encoder
# to the training and testing data
y_department_train = department_encoder.transform(np.array(y_train['Department']).reshape(-1, 1))
y_department_test = department_encoder.transform(np.array(y_test['Department']).reshape(-1, 1))

y_department_train

array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       ...,
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [29]:
# Create a OneHotEncoder for the Attrition column
attrition_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore', drop=None)

# Fit the encoder to the training data
attrition_encoder.fit(np.array(y_train['Attrition']).reshape(-1, 1))

# Create two new variables by applying the encoder
# to the training and testing data
y_attrition_train = attrition_encoder.transform(np.array(y_train['Attrition']).reshape(-1, 1))
y_attrition_test = attrition_encoder.transform(np.array(y_test['Attrition']).reshape(-1, 1))

y_attrition_train

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])

## Create, Compile, and Train the Model

In [31]:
# Find the number of columns in the X training data
features = len(X_train.columns)

# Create the input layer
input = layers.Input((features,),name = 'input')

# Create at least two shared layers
shared1 = layers.Dense(64, activation='relu',name = 'shared1')(input)
shared2 = layers.Dense(128, activation='relu',name = 'shared2')(shared1)

In [32]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_hidden = layers.Dense(32, activation='relu',name = 'department_hidden')(shared2)

# Create the output layer
department_output = layers.Dense(y_department_train.shape[1], activation='sigmoid',name = 'department_output')(department_hidden)


In [33]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden = layers.Dense(32, activation='relu',name = 'attrition_hidden')(shared2)

# Create the output layer
attrition_output = layers.Dense(y_attrition_train.shape[1], activation='sigmoid',name = 'attrition_output')(attrition_hidden)



In [34]:
# Create the model

model = Model(inputs = input, outputs={'department_output': department_output, 'attrition_output': attrition_output})

# Compile the model

model.compile(optimizer='adam',
              loss={
                  'department_output': 'categorical_crossentropy',
                  'attrition_output': 'binary_crossentropy'
              },
              metrics={
                  'department_output': 'accuracy',
                  'attrition_output': 'accuracy'
              })
# Summarize the model
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input (InputLayer)        │ (None, 10)             │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ shared1 (Dense)           │ (None, 64)             │            704 │ input[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ shared2 (Dense)           │ (None, 128)            │          8,320 │ shared1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition_hidden (Dense)  │ (None, 32)             │          4,128 │ shared2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ department_hidden (Dense) │ (None, 32)             │          4,128 │ shared2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition_output (Dense)  │ (None, 2)              │             66 │ attrition_hidden[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ department_output (Dense) │ (None, 3)              │             99 │ department_hidden[0][… │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 17,445 (68.14 KB)

 Trainable params: 17,445 (68.14 KB)

 Non-trainable params: 0 (0.00 B)

In [35]:
# Train the model

fit_model = model.fit(X_train_scaled, {'department_output': y_department_train, 'attrition_output': y_attrition_train}, epochs=100)


Epoch 1/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - attrition_output_accuracy: 0.6765 - attrition_output_loss: 0.6128 - department_output_accuracy: 0.5780 - department_output_loss: 0.9446 - loss: 1.5574
Epoch 2/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_output_accuracy: 0.8411 - attrition_output_loss: 0.4294 - department_output_accuracy: 0.6460 - department_output_loss: 0.7811 - loss: 1.2110
Epoch 3/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.8323 - attrition_output_loss: 0.4079 - department_output_accuracy: 0.6383 - department_output_loss: 0.7674 - loss: 1.1747
Epoch 4/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - attrition_output_accuracy: 0.8463 - attrition_output_loss: 0.3891 - department_output_accuracy: 0.6481 - department_output_loss: 0.7629 - loss: 1.1517
Epoch 5/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - attrition_output_accuracy: 0.8563 - attrition_output_loss: 0.3638 - department_output_accuracy: 0.6596 - department_output_loss: 0

In [36]:
# Evaluate the model with the testing data
results = model.evaluate(X_test_scaled,{'department_output': y_department_test, 'attrition_output': y_attrition_test})
results

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_output_accuracy: 0.8025 - attrition_output_loss: 1.3142 - department_output_accuracy: 0.5015 - department_output_loss: 2.8446 - loss: 4.1679  


[3.780583620071411,
 1.149388313293457,
 2.572763681411743,
 0.8125,
 0.5054348111152649]

In [37]:
# Print the accuracy for both department and attrition
print(f"Department predictions accuracy: {results[4]}")
print(f"Attrition predictions accuracy:{results[3]}")

Department predictions accuracy: 0.5054348111152649
Attrition predictions accuracy:0.8125


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1.
2.
3.